In [1]:
f = lambda x: x**2

# def f(x):
#     return x**2

print(f(5))

25


In this notebook, I create a minimum working example for the `Variable` class of the Gradients Project. Let's first start with a single-input-variable version:

In [5]:
class VariableSingleInput():
    def __init__(self, evaluate=None) :
        if evaluate == None:
            self.evaluate = lambda value: value
        else:
            self.evaluate = evaluate
        
    def __add__(self, other):
        if isinstance(other, (int, float)):
            return VariableSingleInput(evaluate = lambda value: self.evaluate(value) + other)
        
        return VariableSingleInput(evaluate = lambda value: self.evaluate(value) + other.evaluate(value))
        
    def __radd__(self, other):
        return self.__add__(other)


Notice the use of `lambda` to define a function in one line. That makes it much easier than having an incredibly huge `evaluate` method with a ton of different `if`-`else` statements. It would be completely intractible to keep tract of.

Let's test it out:

In [6]:
x = VariableSingleInput()

y = x + 3   # equivalent to running y = x.__add__(3)
print("should be 4:", y.evaluate(1))

# This one proves that the evaluate method works
z = y + x 

print("should be 5:", z.evaluate(1))

should be 4: 4
should be 5: 5


Now, I'll show how to make it take in multidimensional inputs. The key is to use a dictionary that has a "name" for every independent variable.

In [31]:
class Variable():
    def __init__(self, name=None, evaluate=None):
        if evaluate == None:
            self.evaluate = lambda values: values[self.name]
        else:
            self.evaluate = evaluate
            
        if name != None:
            self.name = name          # its key in the evaluation dictionary
    
    def __add__(self, other):
        if isinstance(other, (int, float)):
            return Variable(evaluate = lambda values: self.evaluate(values) + other)
            
        return Variable(evaluate = lambda values: self.evaluate(values) + other.evaluate(values))
        
x_1 = Variable(name="x_1")
x_2 = Variable(name="x_2")

y = x_1 + x_2    # equivalent to running y = x_1.__add__(x_2)
print("should be 4:", y.evaluate({"x_1": 1, "x_2": 3}))

z = y + 2 + x_1 

print("should be 7:", z.evaluate({"x_1": 1, "x_2": 3}))

should be 4: 4
should be 7: 7


In [35]:
# OOP Version

class Variable():
    def __init__(self, name=None):
        if name != None:
            self.name = name          # its key in the evaluation dictionary
            
    def evaluate(self, values):
        """
        This is the evaluate function for independent variables
        """
        return values[self.name]
        
    def __add__(self, other):
        return AdditionVariable(self, other)
    
    

class AdditionVariable(Variable):
    def __init__(self, left, right):
        self.left = left
        self.right = right
        
    def evaluate(self, values):
        if isinstance(self.left, (float, int)):
            return self.right.evaluate(values) + self.left
        
        if isinstance(self.right, (float, int)):
            return self.left.evaluate(values) + self.right
            
        return self.left.evaluate(values) + self.right.evaluate(values)
    

x_1 = Variable(name="x_1")
x_2 = Variable(name="x_2")

y = x_1 + x_2    # equivalent to running y = x_1.__add__(x_2)
print("should be 4:", y.evaluate({"x_1": 1, "x_2": 3}))

z = (y + 2) + x_1  # (y + 2).__add__(x_1)

print("should be 7:", z.evaluate({"x_1": 1, "x_2": 3}))

should be 4: 4
should be 7: 7


## The OOP Version

In [28]:
class VariableOOP():    
    def __init__(self, name=None):            
        self.name = name          # its key in the evaluation dictionary, if it exists
        
    def evaluate(self, values):
        """
        This is the version for independent variables. Override this for other node types
        """
        return values[self.name]
    
    def __add__(self, other):
        return AdditionVariableOOP(left=self, right=other)
    
class AdditionVariableOOP(VariableOOP):
    def __init__(self, name=None, left=None, right=None):
        """
        name should always be "None" here, because these aren't independent variables
        """
        self.left = left
        self.right = right
        
    def evaluate(self, values):
        if isinstance(self.left, (int, float)):
            left = self.left
        else:
            left = self.left.evaluate(values)
        
        if isinstance(self.right, (int, float)):
            right = self.right
        else:
            right = self.right.evaluate(values)
        
        return left + right

x_1 = VariableOOP(name = "x_1")
y = x_1 + 3

print(y.evaluate({"x_1": 1}))

x_2 = VariableOOP(name = "x_2")
z = (x_1 + y) + x_2

print(z.evaluate({"x_1": 3, "x_2": 6}))

4
15


In [29]:
class VariableOOP():
    number_of_independent_variables = 0      # This is needed to create gradients
    
    def __init__(self, name=None):            
        self.name = name          # its key in the evaluation dictionary, if it exists
        if name != None:
            self.position = VariableOOP.number_of_independent_variables
            VariableOOP.number_of_independent_variables += 1
        
    def evaluate(self, values):
        """
        This is the version for independent variables. Override this for other node types
        """
        return values[self.name]
    
    def gradient(self, values):
        """
        This is the version for independent variables. Override this for other node types
        """
        output = np.zeros(VariableOOP.number_of_independent_variables)
        output[self.position] = 1
        return output
    
    def __add__(self, other):
        return AdditionVariableOOP(left=self, right=other)
    
class AdditionVariableOOP(VariableOOP):
    def __init__(self, name=None, left=None, right=None):
        """
        name should always be "None" here, because these aren't independent variables
        """
        self.left = left
        self.right = right
        
    def evaluate(self, values):
        if isinstance(self.left, (int, float)):
            left = self.left
        else:
            left = self.left.evaluate(values)
        
        if isinstance(self.right, (int, float)):
            right = self.right
        else:
            right = self.right.evaluate(values)
        
        return left + right
    
    def gradient(self, values):
        if isinstance(self.left, (int, float)):
            return self.right.gradient(values)
        elif isinstance(self.right, (int, float)):
            return self.left.gradient(values)
        
        return self.left.gradient(values) + self.right.gradient(values)
    

x_1 = VariableOOP(name = "x_1")
y = x_1 + 3

print(y.evaluate({"x_1": 1}))

x_2 = VariableOOP(name = "x_2")
z = (x_1 + y) + x_2

print(z.evaluate({"x_1": 3, "x_2": 6}))
print(z.gradient({"x_1": 3, "x_2": 6}))

4
15
[2. 1.]
